# Face Recognition - Generate Eigenface Classifiers

This notebook tests face detection and recognition from sample images

In [22]:
import cv2, cv2.face
import os, os.path
import numpy as np
from PIL import Image

In [23]:
# constants
CASCADE_PATH = './data/'
CASCADE_NAME = 'haarcascade_frontalface_default.xml'
SAMPLE_TRAINING = './data/sample_training/'
SAMPLE_TESTING = './data/sample_testing/'

In [24]:
#vSAMPLE_TRAINING = '/home/rico/datasets/lfw/raw/'

In [25]:
# Viola-Jones classifier for Haar feature extraction
cascader_file = os.path.join(CASCADE_PATH, CASCADE_NAME)
cascader = cv2.CascadeClassifier(cascader_file)
assert(not cascader.empty())

cascader_kwargs = { # setup detection args here
    'scaleFactor': 1.1,
    'minNeighbors': 2,
    'flags': 2
}

In [26]:
from util import get_images_and_labels

In [27]:
# setup training set
training_set = get_images_and_labels(SAMPLE_TRAINING,
                                    cascader=None,
                                    debug=True,
                                    debug_accuracy=False)

Hosni_Mubarak
	Hosni_Mubarak_0007.png [1 face]
	Hosni_Mubarak_0003.png [1 face]
	Hosni_Mubarak_0004.png [1 face]
	Hosni_Mubarak_0008.png [1 face]
	Hosni_Mubarak_0005.png [1 face]
	Hosni_Mubarak_0001.png [1 face]
	Hosni_Mubarak_0006.png [1 face]
	Hosni_Mubarak_0002.png [1 face]
[Added 8 images]
Jeong_Se-hyun
	Jeong_Se-hyun_0007.png [1 face]
	Jeong_Se-hyun_0001.png [1 face]
	Jeong_Se-hyun_0008.png [1 face]
	Jeong_Se-hyun_0003.png [1 face]
	Jeong_Se-hyun_0004.png [1 face]
	Jeong_Se-hyun_0002.png [1 face]
	Jeong_Se-hyun_0005.png [1 face]
	Jeong_Se-hyun_0006.png [1 face]
[Added 16 images]
Aaron_Peirsol
	Aaron_Peirsol_0003.png [1 face]
	Aaron_Peirsol_0001.png [1 face]
	Aaron_Peirsol_0002.png [1 face]
[Added 19 images]
Heizo_Takenaka
	Heizo_Takenaka_0002.png [1 face]
	Heizo_Takenaka_0007.png [1 face]
	Heizo_Takenaka_0001.png [1 face]
	Heizo_Takenaka_0006.png [1 face]
	Heizo_Takenaka_0004.png [1 face]
	Heizo_Takenaka_0008.png [1 face]
	Heizo_Takenaka_0003.png [1 face]
	Heizo_Takenaka_0005.png 

In [28]:
# setup testing set
testing_set_cascade = get_images_and_labels(SAMPLE_TESTING,
                                    cascader=cascader,
                                    debug=False,
                                    debug_faces=True,
                                    debug_accuracy=True)

testing_set= get_images_and_labels(SAMPLE_TESTING,
                                    cascader=None,
                                    debug=False,
                                    debug_faces=True,
                                    debug_accuracy=True)


Accuracy: 100.00000%
Accuracy: 100.00000%


In [29]:
# setup labels
label_dict = {}
label_dict_inv = {}

for label in training_set[1]:
    if label not in label_dict:
        label_dict_inv[len(label_dict)] = label
        label_dict[label] = len(label_dict)
        

In [30]:
def perform_training(recognizer):
    images, raw_labels = training_set
    labels = [label_dict[label] for label in raw_labels]
    print(labels)
    recognizer.train(images, np.array(labels))
    print('Trained %d images\n' % len(images))

In [31]:
def perform_testing(recognizer, testing_set=testing_set):
    
    images, labels = testing_set
    
    print('Performing %d tests\n' % len(images))

    correct = 0
    total = 0

    for image, expected in zip(images, labels):
        
        if expected not in label_dict:
            print('No face found for %s' % expected)
            
        else:
            eid = label_dict[expected]
            lid, confidence = recognizer.predict(image)
            
            if lid in label_dict_inv:
                
                actual = label_dict_inv[lid]
                                        
                if eid == lid:
                    print('%s is correctly recognized with confidence %.10f' % (expected, confidence))
                    correct += 1
                                        
                else:
                    print('%s incorrect (recognized as %s with confidence %.10f)' % (expected, actual, confidence))
                    
            else:
                print('No face found for %s' % expected)
                
        total += 1

    print()
    print('Accuracy: %.2f%%' % (100 * (correct / max(1, total))))
    print('Correct:  %d' % correct)
    print('Wrong:    %d' % (total - correct))

In [32]:
reco = cv2.face.createLBPHFaceRecognizer()
perform_training(reco)
ims, ls = testing_set
cor = 0
tot = 0
for im, l in zip(ims,ls):
    label = label_dict[l]
    inf, conf = reco.predict(im)
    print(label, inf, conf)
    if (label == inf):
        cor += 1
    tot += 1
print(100*cor/tot,'% Accuracy')

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
Trained 75 images

0 0 90.61975813513436
1 4 88.20881523395161
2 9 86.88206023886762
3 3 81.46586689460405
4 5 78.1746142245438
5 5 74.39190917673132
6 6 78.77813443692419
7 1 84.35208276927965
8 8 80.90414685856982
9 8 91.0965599830775
50.0 % Accuracy


# 1. EigenFaces

In [33]:
# face recognizer
recognizer = cv2.face.createEigenFaceRecognizer()

In [34]:
%%time
# Perform the training
perform_training(recognizer)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
Trained 75 images

CPU times: user 444 ms, sys: 8 ms, total: 452 ms
Wall time: 452 ms


In [35]:
perform_testing(recognizer)

Performing 10 tests

Hosni_Mubarak incorrect (recognized as Hugh_Grant with confidence 7459.1875506163)
Jeong_Se-hyun incorrect (recognized as Jesse_Jackson with confidence 9030.5332386143)
Aaron_Peirsol incorrect (recognized as Jeong_Se-hyun with confidence 7636.2160378138)
Heizo_Takenaka is correctly recognized with confidence 7333.0673881077
Jesse_Jackson incorrect (recognized as George_Clooney with confidence 7408.4107147477)
Hugh_Grant is correctly recognized with confidence 7014.1241709433
Fernando_Gonzalez is correctly recognized with confidence 7018.8116203323
George_Clooney is correctly recognized with confidence 8133.9758120250
Colin_Farrell incorrect (recognized as Hugh_Grant with confidence 7738.7194345377)
Charles_Taylor incorrect (recognized as Fernando_Gonzalez with confidence 9505.4375713488)

Accuracy: 40.00%
Correct:  4
Wrong:    6


# 2. FisherFaces

In [36]:
# face recognizer
recognizer = cv2.face.createFisherFaceRecognizer()

In [37]:
%%time
perform_training(recognizer)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
Trained 75 images

CPU times: user 380 ms, sys: 4 ms, total: 384 ms
Wall time: 381 ms


In [38]:
%%time
perform_testing(recognizer)

Performing 10 tests

Hosni_Mubarak incorrect (recognized as Hugh_Grant with confidence 1412.2747576524)
Jeong_Se-hyun is correctly recognized with confidence 1581.9404932156
Aaron_Peirsol incorrect (recognized as Hosni_Mubarak with confidence 1262.2812054074)
Heizo_Takenaka incorrect (recognized as George_Clooney with confidence 2162.6355459442)
Jesse_Jackson is correctly recognized with confidence 1332.2098677064
Hugh_Grant incorrect (recognized as George_Clooney with confidence 1577.8436376371)
Fernando_Gonzalez is correctly recognized with confidence 1639.7900214552
George_Clooney is correctly recognized with confidence 1633.6195051576
Colin_Farrell incorrect (recognized as Fernando_Gonzalez with confidence 1407.6192953259)
Charles_Taylor incorrect (recognized as Colin_Farrell with confidence 1676.5825641158)

Accuracy: 40.00%
Correct:  4
Wrong:    6
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 8.61 ms


# 2. Local Binary Patterns

In [39]:
# face recognizer
recognizer = cv2.face.createLBPHFaceRecognizer()

In [40]:
%%time
perform_training(recognizer)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
Trained 75 images

CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 192 ms


In [41]:
%%time
perform_testing(recognizer, testing_set)

Performing 10 tests

Hosni_Mubarak is correctly recognized with confidence 90.6197581351
Jeong_Se-hyun incorrect (recognized as Jesse_Jackson with confidence 88.2088152340)
Aaron_Peirsol incorrect (recognized as Charles_Taylor with confidence 86.8820602389)
Heizo_Takenaka is correctly recognized with confidence 81.4658668946
Jesse_Jackson incorrect (recognized as Hugh_Grant with confidence 78.1746142245)
Hugh_Grant is correctly recognized with confidence 74.3919091767
Fernando_Gonzalez is correctly recognized with confidence 78.7781344369
George_Clooney incorrect (recognized as Jeong_Se-hyun with confidence 84.3520827693)
Colin_Farrell is correctly recognized with confidence 80.9041468586
Charles_Taylor incorrect (recognized as Colin_Farrell with confidence 91.0965599831)

Accuracy: 50.00%
Correct:  5
Wrong:    5
CPU times: user 80 ms, sys: 0 ns, total: 80 ms
Wall time: 75.9 ms
